In [31]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Sem 3/AI ML/Lab/Decission_tree/diabetes_data.csv')
df.head()

,glucose,bloodpressure,diabetes
0,86,104,1
1,78,111,0
2,79,114,0
3,77,105,0
4,90,100,1


In [32]:

def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts))
                        for i in range(len(classes))])
    return entropy_value

In [33]:

def calculate_information_gain(dataset,feature,label):

    dataset_entropy = calculate_entropy(dataset[label])
    values,feat_counts= np.unique(dataset[feature],return_counts=True)

    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

In [34]:
def create_decision_tree(dataset, original_dataset, features, label, parent=None):
    unique_labels = np.unique(dataset[label])
    if len(unique_labels) == 1:
        return unique_labels[0]

    elif len(dataset) == 0:
        return np.unique(original_dataset[label])[np.argmax(np.unique(original_dataset[label], return_counts=True)[1])]

    elif len(features) == 0:
        return parent

    else:
        parent = np.unique(dataset[label])[np.argmax(np.unique(dataset[label], return_counts=True)[1])]
        info_gains = [calculate_information_gain(dataset, feature, label) for feature in features]
        best_feature_index = np.argmax(info_gains)
        best_feature = features[best_feature_index]

        tree = {best_feature: {}}

        features = [f for f in features if f != best_feature]

        for value in np.unique(dataset[best_feature]):
            sub_data = dataset.where(dataset[best_feature] == value).dropna()
            subtree = create_decision_tree(sub_data, original_dataset, features, label, parent)
            tree[best_feature][value] = subtree

        return tree

In [35]:
def predict_diabetes(sample_data, tree):
    for node in tree.keys():
        value = sample_data[node]
        tree = tree[node][value]
        prediction = tree
        if isinstance(tree, dict):
            prediction = predict_diabetes(sample_data, tree)
    return prediction

features = df.columns[:-1]
label = 'diabetes'
parent = None
features

Index(['glucose', 'bloodpressure'], dtype='object')

In [36]:
decision_tree = create_decision_tree(df,df,features,label,parent)

In [37]:
# Predict using the trained model
sample_data = {'glucose':86,'bloodpressure':104}
test_data = pd.Series(sample_data)
prediction = predict_diabetes(test_data,decision_tree)
prediction

1.0

# Method 2

In [ ]:
import numpy
import pandas
import random

def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

def checkPurity(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False

def classifyData(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]

def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in range(len(uniqueValues)):
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits

def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]

def calculateEntropy(data):
    _, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()
    return sum(probabilities * -numpy.log2(probabilities))

def calculateOverallEntropy(dataBelow, dataAbove):
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)

def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in range(randomSplits):
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue

def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree

def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)

def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

In [ ]:
import pandas as pd
import numpy as np

# Create a sample dataset
data = {
    'glucose': [86, 78, 79, 77, 90, 100, 105, 110, 115, 120],
    'bloodpressure': [104, 111, 114, 105, 100, 95, 90, 85, 80, 75],
    'diabetes': [1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
}

df = pd.DataFrame(data)

# Train the decision tree
decision_tree = buildDecisionTree(df, randomAttributes=2, randomSplits=10)

# Print the decision tree
print(decision_tree)

# Make predictions on the training data
predictions = decisionTreePredictions(df, decision_tree)

# Print the predictions
print(predictions)

# Calculate the accuracy of the predictions
accuracy = calculateAccuracy(predictions, df['diabetes'])
print("Accuracy:", accuracy)

{'bloodpressure <= 104.5': [1, 0]}
0    1
1    0
2    0
3    0
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
Accuracy: 1.0
